# Overview of MNE-Python's data structures

In [76]:
import mne
import os.path as op
from __future__ import print_function
#import numpy as np

## `Info`: recording information
The `Info` data object is typically created when data is imported into MNE-Python and contains details such as:

 - date, subject information, and other recording details
 - the samping rate
 - information about the data channels (name, type, position, etc.)
 - digitized points
 - sensor–head coordinate transformation matrices

and so forth. See the [reference documentation](http://martinos.org/mne/stable/manual/matlab.html) for a complete list of all data fields. Once created, this object is passed around throughout the data analysis pipeline.

It behaves as a nested Python dictionary:

In [77]:
# Read the info object from an example recording 
info = mne.io.read_info(op.join(mne.datasets.sample.data_path(), 'MEG', 'sample', 'sample_audvis_raw.fif'), verbose=False)

# List all the fields in the info object
print('Keys in info dictionary:\n', info.keys())

Keys in info dictionary:
 ['acq_stim', 'lowpass', 'description', 'dev_ctf_t', 'projs', 'meas_date', 'meas_id', 'subject_info', 'sfreq', 'chs', 'events', 'dev_head_t', 'line_freq', 'proj_id', 'hpi_results', 'highpass', 'hpi_subsystem', 'comps', 'custom_ref_applied', 'experimenter', 'file_id', 'proj_name', 'acq_pars', 'nchan', 'bads', 'hpi_meas', 'dig', 'ctf_head_t', 'ch_names']


In [78]:
# Obtain the sampling rate of the data
print(info['sfreq'], 'Hz')

600.614990234 Hz


In [79]:
# List all information about the first data channel
print(info['chs'][0])

{'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.0127    ,  0.0057    ,
       -0.99990302, -0.186801  , -0.98240298, -0.0033    , -0.98232698,
        0.18674099,  0.013541  ], dtype=float32), 'kind': 1, 'eeg_loc': None, 'ch_name': 'MEG 0113', 'coil_trans': array([[-0.0127    , -0.186801  , -0.98232698, -0.1066    ],
       [ 0.0057    , -0.98240298,  0.18674099,  0.0464    ],
       [-0.99990302, -0.0033    ,  0.013541  , -0.0604    ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), 'coord_frame': 1, 'range': 0.00030517578125, 'unit_mul': 0, 'unit': 201, 'cal': 3.1600000394149674e-09, 'coil_type': 3012, 'scanno': 1, 'logno': 113}


## Obtaining subsets of channels

There are a number of convenience functions to obtain channel indices, given an `Info` object.

In [80]:
# Get channel indices by name
channel_indices = mne.pick_channels(info['ch_names'], ['MEG 0312', 'EEG 005'])

# Get channel indices by regular expression
channel_indices = mne.pick_channels_regexp(info['ch_names'], 'MEG *')

# Get channel indices by type
channel_indices = mne.pick_types(info, meg=True) # MEG only
channel_indices = mne.pick_types(info, eeg=True) # EEG only
channel_indices = mne.pick_types(info, meg='grad', eeg=True) # MEG gradiometers and EEG channels

# Get a dictionary of channel indices, grouped by channel type
channel_indices_by_type = mne.io.pick.channel_indices_by_type(info)
print('The first three magnetometers:', channel_indices_by_type['mag'][:3])

The first three magnetometers: [2, 5, 8]


## Obtaining information about channels

In [85]:
# Channel type of a specific channel
channel_type = mne.io.pick.channel_type(info, 75)
print('Channel #75 is of type:', channel_type)

# Channel types of a collection of channels
meg_channels = mne.pick_types(info, meg=True)[:10]
channel_types = [mne.io.pick.channel_type(info, ch) for ch in meg_channels]
print('First 10 MEG channels are of type:\n', channel_types)


Channel #75 is of type: grad
First 10 MEG channels are of type:
 ['grad', 'grad', 'mag', 'grad', 'grad', 'mag', 'grad', 'grad', 'mag', 'grad']


## Dropping channels from an info structure

It is possible to limit the info structure to only include a subset of channels with the `pick_info` function:

In [82]:
# Only keep EEG channels
eeg_indices = mne.pick_types(info, meg=False, eeg=True)
reduced_info = mne.pick_info(info, eeg_indices)

print(reduced_info)

<Info | 23 non-empty fields
    acq_pars : unicode | 13886 items
    bads : list | 0 items
    ch_names : list | EEG 001, EEG 002, EEG 003, EEG 004, EEG 005, EEG 006, ...
    chs : list | 59 items (EEG: 59)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : unicode | 49 items
    dev_head_t : dict | 3 items
    dig : list | 146 items
    events : list | 1 items
    experimenter : unicode | 3 items
    file_id : dict | 4 items
    highpass : float | 0.10000000149
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    lowpass : float | 172.176300049
    meas_date : numpy.ndarray | 2002-12-03 20:01:10
    meas_id : dict | 4 items
    nchan : int | 59
    proj_id : numpy.ndarray | 1 items
    proj_name : unicode | 4 items
    projs : list | PCA-v1: off, PCA-v2: off, PCA-v3: off
    sfreq : float | 600.614990234
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    hpi_subsystem : NoneType
    line_freq : NoneType
    subj